In [1]:
import pandas as pd
import numpy as np
import re
import string

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk
import spacy

2023-07-04 22:11:18.165841: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-04 22:11:18.233444: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-04 22:11:18.598969: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-04 22:11:18.600775: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 22:11:21.544539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [ ]:
# nltk.download()
! pip install bs4 nltk spacy gensim
! python -m spacy download en_core_web_sm 

In [2]:
labeledTrainData_df = pd.read_table("../Dataset/labeledTrainData.tsv",index_col=None)
sentiments = labeledTrainData_df['sentiment'].to_numpy()
reviews_unclean = labeledTrainData_df['review'].to_numpy()

In [3]:
CLEANR = re.compile('<.*?>') 
escapes = ''.join([chr(char) for char in range(1, 32)])
translator1 = str.maketrans('', '', '.!"#$%&\()*+,-/:;<=>?@[\\]^_`{|}~') #removing '
translator2 = str.maketrans('', '', '!"#$%&\()*+,-/:;<=>?@[\\]^_`{|}~') #removing . and ' 
sp = spacy.load('en_core_web_sm')


In [16]:
def remove_html_tags(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

def clean(revs_unclean):
    for review in revs_unclean:
        
        # rev = remove_html_tags(review).translate(translator2).replace("n't"," not").lower()
        rev = remove_html_tags(review).translate(translator2).lower()
        rev_sen = sp(rev)
        rev_clean = []
        
        for idx,word in enumerate(rev_sen):

            if (word.pos_ == "NOUN"  and (idx == 0 or  rev_sen[idx-1].pos_ not in ["ADJ","ADV"]) ):
                continue
            
            # if (word.pos_  in ["VERB","ADJ","ADV","INTJ"] and word.text not in string.punctuation and not word.is_stop ) or (word.pos_ == "PART" and word.text == "not"):  
            if (word.pos_  in ["VERB","ADJ","ADV","INTJ"] and word.text not in string.punctuation and not word.is_stop ):  
                rev_clean.append(word.lemma_)
        
        
        yield rev_clean

from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from textblob import TextBlob

negation_words = ["not","nor","none","never","neither"]

def clean_review(revs_unclean):

    stops = set(stopwords.words("english")) - set(negation_words)
    is_noun = lambda pos: pos[:2] == 'NN' and len(pos) >2

    for review in revs_unclean:
        review_text = BeautifulSoup(review).get_text()
        review_text = re.sub("[^a-zA-Z]"," ",review_text)
        blob = TextBlob(review_text)
        
        words = [wordnet_lemmatizer.lemmatize(w.lower()) for w,pos in blob.pos_tags if not is_noun(pos) and w.lower() not in stops and w not in string.punctuation]
        yield words


        

        
        



In [17]:
cleaner = clean_review(reviews_unclean)
# print(next(cleaner))
all_words = []
review_clean = []
for idx,review in enumerate(cleaner):
    if idx%1000 == 0:
        print("\f\r{}".format(idx//1000 + 1),end="")
    all_words += review
    review_clean.append(review)

import collections
counter = collections.Counter(all_words)
freq = dict(counter)


1

/tmp/ipykernel_3571/3648827142.py:37: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


25

# Method 1

In [93]:
import operator
def occurrence(word_list,word):
    return operator.countOf(word_list,word)

def generate_vector(revs_clean,limit,freq):
    words = [k for k, v in sorted(freq.items(), key=lambda item: item[1])][::-1][:limit]
    reviews_vec = []
    for idx,review in enumerate(revs_clean):
        vec = [occurrence(review,word) for word in words]
        reviews_vec.append(vec)
    return reviews_vec
vector_of_reviews = generate_vector(review_clean,1000,freq)
        

# Write Code Here (Method 2)

In [ ]:
# Get word relation vectors of word in all_words
# from the top 500 frequenct one choose there 5 best similar meaning word from it and make array word_vec_2
# then convert the review_clean based on that

In [17]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "../Dataset/labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "../Dataset/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "../Dataset/unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print ("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [35]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import 	WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z']"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    words = [wordnet_lemmatizer.lemmatize(w) for w in words]
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [36]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [37]:
sentences = []  # Initialize an empty list of sentences

print( "Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print( "Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/tmp/ipykernel_3577/3457430185.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
/tmp/ipykernel_3577/3457430185.py:14: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


Parsing sentences from unlabeled set


In [1]:
# import nltk
# from nltk.stem import 	WordNetLemmatizer
# wordnet_lemmatizer = WordNetLemmatizer()
# text = "studies studi cries cry"
# tokenization = nltk.word_tokenize(text)
# for w in tokenization:
#     print("Lemma for {} is {}".format(w, wordnet_lemmatizer.lemmatize(w))) 

Lemma for studies is study
Lemma for studi is studi
Lemma for cries is cry
Lemma for cry is cry


In [38]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='\f\r%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 500    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 25         # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
epoch =5

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling,epochs=epoch)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2023-07-04 19:35:30,595 : INFO : collecting all words and their counts
2023-07-04 19:35:30,597 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-07-04 19:35:30,646 : INFO : PROGRESS: at sentence #10000, processed 221389 words, keeping 17034 word types
2023-07-04 19:35:30,685 : INFO : PROGRESS: at sentence #20000, processed 443163 words, keeping 24221 word types
2023-07-04 19:35:30,732 : INFO : PROGRESS: at sentence #30000, processed 658078 words, keeping 29380 word types
2023-07-04 19:35:30,779 : INFO : PROGRESS: at sentence #40000, processed 879887 words, keeping 33856 word types


Training model...


2023-07-04 19:35:30,830 : INFO : PROGRESS: at sentence #50000, processed 1095020 words, keeping 37475 word types
2023-07-04 19:35:30,893 : INFO : PROGRESS: at sentence #60000, processed 1312318 words, keeping 40676 word types
2023-07-04 19:35:30,948 : INFO : PROGRESS: at sentence #70000, processed 1530945 words, keeping 43512 word types
2023-07-04 19:35:31,003 : INFO : PROGRESS: at sentence #80000, processed 1746080 words, keeping 46170 word types
2023-07-04 19:35:31,060 : INFO : PROGRESS: at sentence #90000, processed 1965830 words, keeping 48868 word types
2023-07-04 19:35:31,145 : INFO : PROGRESS: at sentence #100000, processed 2183384 words, keeping 51154 word types
2023-07-04 19:35:31,198 : INFO : PROGRESS: at sentence #110000, processed 2398219 words, keeping 53242 word types
2023-07-04 19:35:31,245 : INFO : PROGRESS: at sentence #120000, processed 2616264 words, keeping 55522 word types
2023-07-04 19:35:31,287 : INFO : PROGRESS: at sentence #130000, processed 2837604 words, keep

In [8]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load("300features_40minwords_10context")

In [9]:
model.wv.most_similar("good",topn=100)

[('decent', 0.6503987312316895),
 ('great', 0.5574769973754883),
 ('nice', 0.5495736598968506),
 ('bad', 0.5347516536712646),
 ('cool', 0.5061217546463013),
 ('passable', 0.4860948622226715),
 ('lousy', 0.480047345161438),
 ('alright', 0.47417131066322327),
 ('ok', 0.4614225924015045),
 ('mediocre', 0.4576554298400879),
 ('solid', 0.4550914764404297),
 ('okay', 0.4472653269767761),
 ('fine', 0.4454362988471985),
 ('watchable', 0.4410836696624756),
 ('weak', 0.4355188310146332),
 ('disappointing', 0.4158312678337097),
 ('terrible', 0.4102785289287567),
 ('interesting', 0.4023219347000122),
 ('funny', 0.4009913206100464),
 ('crappy', 0.3882032632827759),
 ('impressive', 0.3880762457847595),
 ('terrific', 0.3867708444595337),
 ('fair', 0.3807121813297272),
 ('excellent', 0.3805137574672699),
 ('awesome', 0.3800690472126007),
 ('promising', 0.3793089687824249),
 ('entertaining', 0.3726573586463928),
 ('cheesy', 0.37158483266830444),
 ('reasonable', 0.3633931875228882),
 ('uneven', 0.360927

In [18]:
[k for k, v in sorted(freq.items(), key=lambda item: item[1])][::-1]

['movie',
 'film',
 'not',
 'one',
 'like',
 'good',
 'even',
 'time',
 'would',
 'get',
 'make',
 'see',
 'story',
 'really',
 'much',
 'well',
 'also',
 'bad',
 'great',
 'made',
 'first',
 'way',
 'could',
 'think',
 'show',
 'go',
 'know',
 'character',
 'two',
 'life',
 'watch',
 'seen',
 'say',
 'many',
 'plot',
 'end',
 'love',
 'acting',
 'never',
 'little',
 'look',
 'best',
 'ever',
 'better',
 'take',
 'still',
 'scene',
 'come',
 'work',
 'man',
 'find',
 'something',
 'want',
 'give',
 'back',
 'real',
 'watching',
 'thing',
 'though',
 'another',
 'old',
 'play',
 'funny',
 'actually',
 'going',
 'nothing',
 'lot',
 'director',
 'cast',
 'part',
 'quite',
 'got',
 'around',
 'seems',
 'every',
 'pretty',
 'however',
 'fact',
 'thought',
 'enough',
 'feel',
 'u',
 'original',
 'world',
 'series',
 'long',
 'horror',
 'young',
 'action',
 'without',
 'role',
 'right',
 'point',
 'must',
 'always',
 'may',
 'comedy',
 'least',
 'saw',
 'interesting',
 'done',
 'new',
 'whole

In [19]:
import operator
def occurrence_of_list(sen,wordlist):
    count = 0
    for word in wordlist:
        count += operator.countOf(sen,word)
    return count
def generate_word_list(words):
    wordlist = []
    for word in words:
        templist = [word]
        try:
            newlist = np.array(model.wv.most_similar(word,topn=10))[:,0].tolist()
        except:
            print("I am in Exception",word)
            newlist = []
        wordlist.append(templist + newlist)
    return wordlist
def convert_to_vector(rev_clean,freq,limit):
    words = [k for k, v in sorted(freq.items(), key=lambda item: item[1])][::-1][2:limit+2]
    words_of_word = generate_word_list(words)

    reviews_vec = []
    for idx,review in enumerate(rev_clean):
        vec = [occurrence_of_list(review,word_list) for word_list in words_of_word]
        reviews_vec.append(vec)
    return reviews_vec

vector_of_reviews = convert_to_vector(review_clean,freq,1000)
    

In [15]:
len(vector_of_reviews)

1000

# Test Data

In [25]:

#test data
TestData_df = pd.read_table("../Dataset/testData.tsv",index_col=None)
test_review_unclean = TestData_df['review'].to_numpy()
test_cleaner = clean_review(test_review_unclean)
test_review_clean = []
for idx,review in enumerate(test_cleaner):
    if idx%10 == 0:
        print("\f\r{}".format(idx//10 + 1),end="")
    test_review_clean.append(review)

test_vector_of_reviews = convert_to_vector(test_review_clean,freq,1000)


4

/tmp/ipykernel_3571/3648827142.py:37: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


2500

# DataSplit

In [20]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(vector_of_reviews,sentiments, test_size = 0.25, random_state = 42)
print('Training Features Shape:', np.array(train_features).shape)
print('Training Labels Shape:', np.array(train_labels).shape)
print('Testing Features Shape:', np.array(test_features).shape)
print('Testing Labels Shape:', np.array(test_labels).shape)

Training Features Shape: (18750, 1000)
Training Labels Shape: (18750,)
Testing Features Shape: (6250, 1000)
Testing Labels Shape: (6250,)


# Random Forest

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)# Train the model on training data
rf.fit(train_features, train_labels)
predictions = rf.predict(test_features)
predictions = [1 if p >= 0.5 else 0 for p in predictions]
predictions = np.array(predictions)
test_labels = np.array(test_labels)
accuracy = 1 - np.count_nonzero(test_labels-predictions)/len(test_labels)
print(accuracy)

# SVM

In [60]:
from sklearn import svm
svm_clf = svm.SVC(kernel="linear")
svm_clf.fit(train_features,train_labels)
predictions = svm_clf.predict(test_features)
test_labels = np.array(test_labels)
accuracy = 1 - np.count_nonzero(test_labels-predictions)/len(test_labels)
print(accuracy)

0.8384


# KNN

In [59]:
from sklearn.neighbors import KNeighborsClassifier
# accuracies = []
k=47
knn_clf = KNeighborsClassifier(n_neighbors=k)
knn_clf.fit(train_features, train_labels)
predictions = knn_clf.predict(test_features)

test_labels = np.array(test_labels)
accuracy = 1 - np.count_nonzero(test_labels-predictions)/len(test_labels)
print(accuracy)
# accuracies.append(accuracy)

# from matplotlib import pyplot as plt
# x_axis = range(3,100,2)
# y_axis = accuracies
# plt.plot(x_axis,y_axis)
# plt.show()
# plt.savefig("KNN.png")

0.71088


# Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
# Build a Gaussian Classifier
bayes_clf = LinearDiscriminantAnalysis()
# bayes_clf = GaussianNB()

# Model training
bayes_clf.fit(train_features,train_labels)
predictions = bayes_clf.predict(test_features)

test_labels = np.array(test_labels)
accuracy = 1 - np.count_nonzero(test_labels-predictions)/len(test_labels)
print(accuracy)


0.85344


# Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()

log_clf.fit(train_features,train_labels)
predictions = log_clf.predict(test_features)

test_labels = np.array(test_labels)
accuracy = 1 - np.count_nonzero(test_labels-predictions)/len(test_labels)
print(accuracy)

0.85856


/home/sagittarius/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
predictions = log_clf.predict(test_vector_of_reviews)
ids = TestData_df['id'].to_list()
test_output_df = pd.DataFrame(columns = ["id","sentiment"])

test_output_df['id'] = ids
test_output_df['sentiment'] = predictions

# test_output_df.to_csv("submission.csv",index=False,sep=",",encoding='utf-8')

In [28]:
test_output_df.to_csv("submission.csv",index=False,sep=",",encoding='utf-8')



# Random Forest
# 0.778 when n_estimator = 100
# 0.78096 when n_estimator = 200
# 0.78352 when n_estimator = 1000 (28 min)

# SVM
# 0.83008 when "linear" (11 min)  -> 0.83488 (7 min 13 sec) (Without Noun) -> 0.8384 (15 min) (With 1000 wordsvector)

# KNN
# 0.72064 (15 sec) k=47      -> 0.7184 (7 sec) (Without Noun)  -> 0.71088 (8.3 sec) (With 1000 wordsvector)

# Naive bayes
# 0.78888 (1.4 sec) GaussianNB
# 0.8296 (5.4 sec) LDA        -> 0.8336 (3.7 sec) (Without Noun) -> 0.83488 (8 sec) (With 1000 wordsvector) -> 0.84096 -> 0.83744 (With 20 similar meaning) -> 0.85296 (removing all stop words and taking all part of speech) -> 0.85344 (adding negation words)
# 0.79216 (4.5 sec) QDA

# Logistic regression
# 0.83552 (2.7 sec) -> 0.83984 (2.3 sec) (Without Noun) -> 0.84288 (5.2 sec) (With 1000 wordsvector)  -> 0.8456 (with 10 similar meaning) -> 0.84768 (With 20 similar meaning)  -> 0.8576 (removing all stop words and taking all part of speech) (Method 2) -> 0.85648 (removing all stop words and taking all part of speech) (Method 1)
-> 0.85856 (adding negation words) Method 2 -> 0.8628 (10 epochs) (Method 2) , -> 0.8544 (Method 1)







(Without Lemminization)
# Method 1 
# 0.49472 (1000 features)

In [153]:
sen = sp("wow! its my fault ")
for word in sen:
    print(f'{word.text:{12}} {word.pos_:{10}} {word.tag_:{8}} {spacy.explain(word.tag_)}')# 
    

wow          INTJ       UH       interjection
!            PUNCT      .        punctuation mark, sentence closer
its          PRON       PRP$     pronoun, possessive
my           PRON       PRP$     pronoun, possessive
fault        NOUN       NN       noun, singular or mass
